In [84]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

In [101]:
import zomatopy
import json
import requests
import ast

In [102]:
config={ "user_key":"279658393d35002cb45c944d0dd06691"} #New Generated id added
zomato = zomatopy.initialize_app(config)

In [103]:
loc = 'Delhi'
budget = 'medium'
cuisine = 'Chinese'

In [104]:
location_detail=zomato.get_location(loc, 1)

In [105]:
d1 = json.loads(location_detail)
lat=d1["location_suggestions"][0]["latitude"]
lon=d1["location_suggestions"][0]["longitude"]

In [106]:
d1["location_suggestions"][0]["city_id"]

1

In [107]:
d1["location_suggestions"][0]["city_name"]

'Delhi NCR'

In [108]:
#zomato.get_cuisines(d1["location_suggestions"][0]["city_id"])

In [109]:
cuisines_dict={'Chinese':25,'Mexican':73,'Italian':55,'American':1,'South Indian':85,'North Indian':50}

In [110]:
str(cuisines_dict.get('Chinese'))

'25'

In [143]:
if (budget == 'low'):
        lowercost = 0
        highercost = 300
elif (budget == 'medium'):
        lowercost = 300
        highercost = 700
elif (budget == 'high'):
        lowercost = 700
        highercost = 999999 

In [147]:
  #creating custom function to accomodate sort,order,start,limit
def restaurant_search_custom(query="", latitude="", longitude="", cuisines="", limit=20, start=0, sort="", order=""):
            """
            Takes either query, latitude and longitude or cuisine as input.
            Returns a list of Restaurant IDs.
            """
            cuisines = "%2C".join(cuisines.split(","))
            base_url="https://developers.zomato.com/api/v2.1/"
            if str(limit).isalpha() == True:
                raise ValueError('LimitNotInteger')
            headers = {'Accept': 'application/json', 'user-key': "279658393d35002cb45c944d0dd06691"}
            r = (requests.get(base_url + "search?"+ "count=" + str(limit) + "&lat=" + str(latitude) + "&lon=" + str(longitude) + "&cuisines=" + str(cuisines) + "&sort=" + str(sort) + "&order=" + str(order)+ "&start=" + str(start) , headers=headers).content).decode("utf-8")
            #a = ast.literal_eval(r)
            jsonParsed  = json.loads(r)
            return jsonParsed['restaurants']

In [148]:
    #filtering retaurants by rating, budget
def restraunt_search_rating_order(latitude="", longitude="", cuisines="", lowercost=0, highercost=999999):

            restraunts = []
            start = 0
            sort = "rating"
            order = "desc" 
            while(len(restraunts) < 5):
                    returnList = restaurant_search_custom("", lat, lon, str(cuisines_dict.get(cuisine)), 20,start,sort,order)
                    # iterate list to add items if fall in category
                    for returnItem in returnList:
                            #jsonParsed  = json.loads(returnItem)
                            if((len(restraunts) < 5) and (lowercost <= returnItem['restaurant']['average_cost_for_two']) and (returnItem['restaurant']['average_cost_for_two'] <= highercost)):
                                    restraunts.append(returnItem)
                    start = start + 20                

            return restraunts 

In [149]:
results = restraunt_search_rating_order(lat, lon, str(cuisines_dict.get(cuisine)),lowercost, highercost)
response=""
if len(results) == 0:
    response= "no results"
else:
    for restaurant in results:
        response=response+ "Found "+ restaurant['restaurant']['name'] + " in "+ restaurant['restaurant']['location']['address']+ str(restaurant['rating'])+"\n"
#dispatcher.utter_message("---------"+response)

Found Snacks PointFound Kahi Bhi Kuch BhiFound Global PlattersFound The Curry Grill-TCGFound Urban O Chaska


In [12]:
results=zomato.restaurant_search("", lat, lon, str(cuisines_dict.get("Chinese")), 200)

In [13]:
d = json.loads(results)

In [14]:
restaurant_found_list = []

In [15]:
for restaurant in d['restaurants']:
    restaurant_found_list.append(
        {
            "Restauran_Name" : restaurant['restaurant']['name'],
			"Address" : restaurant['restaurant']['location']['address'],
			"Rating" : restaurant['restaurant']['user_rating']['aggregate_rating'],
			"Budget" : restaurant['restaurant']['average_cost_for_two']
		}
	)

In [16]:
len(restaurant_found_list)

20

In [17]:
budget = 'high'
restaurant_filtered_list = []
for restaurant in restaurant_found_list:
	if ((budget == 'low') and (restaurant['Budget'] < 300)):
		restaurant_filtered_list.append(restaurant)
	elif ((budget == 'medium') and (restaurant['Budget'] >= 300) and (restaurant['Budget'] <= 700)):
		restaurant_filtered_list.append(restaurant)
	elif ((budget == 'high') and (restaurant['Budget'] > 700)):
		restaurant_filtered_list.append(restaurant)

In [18]:
restaurant_filtered_list

[{'Restauran_Name': 'Boa Village',
  'Address': '13, Alipur Road, Civil Lines, New Delhi',
  'Rating': '4.3',
  'Budget': 2300},
 {'Restauran_Name': 'Pa Pa Ya',
  'Address': 'A-3, Select Citywalk, District Centre, Saket, New Delhi',
  'Rating': '4.9',
  'Budget': 2500},
 {'Restauran_Name': 'Plum By Bent Chair',
  'Address': 'Ground Floor, Worldmark 2, Aerocity, New Delhi',
  'Rating': '4.6',
  'Budget': 2200},
 {'Restauran_Name': 'Yeti - The Himalayan Kitchen',
  'Address': 'T 7 A, Khirki Extension, Malviya Nagar, New Delhi',
  'Rating': '4.4',
  'Budget': 1500},
 {'Restauran_Name': 'Enoki - Fresh Asian Kitchen',
  'Address': 'Greater Kailash (GK) 1, New Delhi',
  'Rating': '4.0',
  'Budget': 1000},
 {'Restauran_Name': 'Kylin Experience',
  'Address': 'T-302, 3rd Floor, Ambience Mall, Vasant Kunj, New Delhi',
  'Rating': '4.2',
  'Budget': 1800},
 {'Restauran_Name': 'Noshi - Yum Asian Delivery',
  'Address': '23, Community Centre, Zamrudpur, Greater Kailash 1 (GK 1), New Delhi',
  'Rat

In [19]:
number_of_restaurants_filtered = len(restaurant_filtered_list)
number_of_restaurants_filtered

15

In [20]:
sorted(restaurant_filtered_list, key=lambda x: x['Budget'], reverse=True)

[{'Restauran_Name': 'Pa Pa Ya',
  'Address': 'A-3, Select Citywalk, District Centre, Saket, New Delhi',
  'Rating': '4.9',
  'Budget': 2500},
 {'Restauran_Name': 'Honk - Pullman New Delhi Aerocity',
  'Address': 'Asset 2, GMR Hospitality District, Near IGI Airport, Aerocity, New Delhi',
  'Rating': '4.6',
  'Budget': 2500},
 {'Restauran_Name': 'Boa Village',
  'Address': '13, Alipur Road, Civil Lines, New Delhi',
  'Rating': '4.3',
  'Budget': 2300},
 {'Restauran_Name': 'Kylin Skybar',
  'Address': 'T-302, 3rd Floor, Ambience Mall, Vasant Kunj, New Delhi',
  'Rating': '4.1',
  'Budget': 2300},
 {'Restauran_Name': 'Plum By Bent Chair',
  'Address': 'Ground Floor, Worldmark 2, Aerocity, New Delhi',
  'Rating': '4.6',
  'Budget': 2200},
 {'Restauran_Name': 'Plum By Bent Chair',
  'Address': '22, Janpath Road, Janpath, New Delhi',
  'Rating': '4.4',
  'Budget': 2200},
 {'Restauran_Name': "Master Koii's",
  'Address': 'Shop 19, Flavor Town, Bestech Chambers, Sushant Lok, Gurgaon',
  'Rating

In [21]:
for restaurantdetail in sorted(restaurant_filtered_list, key=lambda x: float(x['Rating']), reverse=True):
    print(restaurantdetail['Restauran_Name'])

Pa Pa Ya
Plum By Bent Chair
Honk - Pullman New Delhi Aerocity
You Mee
Yeti - The Himalayan Kitchen
Master Koii's
Jom Jom Malay
Plum By Bent Chair
Boa Village
Nagaland's Kitchen
Kylin Experience
Noshi - Yum Asian Delivery
Kylin Skybar
Mamagoto
Enoki - Fresh Asian Kitchen
